In [13]:
from __future__ import print_function

# https://qiita.com/kzkadc/items/e4fc7bc9c003de1eb6d0
# argument Parser
import argparse

import os
import random
import math
import torch
import torch.nn as nn
#import torch.legacy.nn as lnn # do not use?
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

# visdom is visualize the process of learning
# torchsummary is network summarizing tool
# both are usefull!
# https://qiita.com/yasudadesu/items/1dda5f9d1708b6d4d923
#
import visdom

from torch.autograd import Variable

In [14]:
# initial setting
opt = {'dataset':'folder',
       'dataroot':'celebA/test/',
       'workers':2,
       'batchSize':64,
       'imageSize':64,
       'nz':100,
       'ngf':64,
       'ndf':64,
       'niter':25,
       'saveInt':25,
       'lr':0.0002,
       'beta1':0.5,
       'cuda':False,
       'ngpu':1,
       #'netG':,
       #'netD':,
       'outf':'./output_model',
       'manualSeed':462528}
print(opt)

{'dataset': 'folder', 'dataroot': 'celebA/test/', 'workers': 2, 'batchSize': 64, 'imageSize': 64, 'nz': 100, 'ngf': 64, 'ndf': 64, 'niter': 25, 'saveInt': 25, 'lr': 0.0002, 'beta1': 0.5, 'cuda': False, 'ngpu': 1, 'outf': './output_model', 'manualSeed': 462528}


In [15]:
try:
    os.makedirs(opt['outf']) # 引数に渡されたディレクトリに出力imageとモデルのチェックポイントを書き込むディレクトリを作る
except OSError:
    pass # もしエラーが発生した場合には何もしない

In [16]:
# 引数にmanualSeedが渡された場合の処理
# https://qiita.com/chat-flip/items/4c0b71a7c0f5f6ae437f
if opt['manualSeed'] is None:
    opt['manualSeed'] = random.randint(1, 10000) # manualSeedがNoneの場合、1~10000の間の一様乱数から整数を取得してseedにする
print("Random Seed: ", opt['manualSeed']) # 設定されたseedをprint
random.seed(opt['manualSeed']) # 乱数のseedにopt.manualSeedを設定
torch.manual_seed(opt['manualSeed']) # 畳み込み層の重みの乱数seedを設定
if opt['cuda']:
    torch.cuda.manual_seed_all(opt['manualSeed']) # GPUを使う場合は別の設定にする

Random Seed:  462528


In [17]:
# オートチューナが最適なアルゴリズムを見つける
# https://qiita.com/koshian2/items/9877ed4fb3716eac0c37
cudnn.benchmark = True

In [18]:
# GPUが利用可能な環境だがopt.cuda引数がtrueでないときにwarningを出す処理
if torch.cuda.is_available() and not opt['cuda']:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

In [19]:
import glob
file = glob.glob(opt['dataroot']+'*')
print(len(file))

10


In [44]:
attr_list = []

# ファイルをオープンする
test_data = open('./celebA/list_attr_celeba_test.txt', "r")

# 一行ずつ読み込んでは表示する
i = 0
for line in test_data:
    if i == 1:
        name = line.split()
    if i >= 2:
        out = line.split()
        attr_list.append(out)
    i += 1

# ファイルをクローズする
test_data.close()

print(attr_list[3][0])

000004.jpg


In [51]:
# ref : https://tips-memo.com/pytorch-dataset
# define transform
transform=transforms.Compose([
    transforms.Resize(opt['imageSize']),
    transforms.CenterCrop(opt['imageSize']),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


from skimage import io
from sklearn.model_selection import train_test_split
from PIL import Image

# define dataset
# https://qiita.com/sheep96/items/0c2c8216d566f58882aa
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, csv_file_path, root_dir, transform=None):
        #pandasでcsvデータの読み出し
        self.image_attr_list = attr_list
        self.root_dir = root_dir
        #画像データへの処理
        self.transform = transform

    def __len__(self):
        return len(self.image_attr_list)

    def __getitem__(self, idx):
        #dataframeから画像へのパスとラベルを読み出す
        #label = self.image_dataframe.iat[idx, LABEL_IDX]
        img_name = os.path.join(self.root_dir,
                                self.image_attr_list[idx][0])
        #画像の読み込み
        image = io.imread(img_name)
        PIL_image = Image.fromarray(image)
        #画像へ処理を加える
        if self.transform:
            image = self.transform(PIL_image)

        return image #,label
    
imgDataset = MyDataset(attr_list, opt['dataroot'], transform=transform)

train_data, test_data = train_test_split(imgDataset, test_size=0.2)

print(train_data)

[tensor([[[-0.5843, -0.5294, -0.4588,  ..., -0.9686, -0.9608, -0.9451],
         [-0.6784, -0.5686, -0.3490,  ..., -0.9765, -0.9686, -0.9451],
         [-0.7569, -0.6078, -0.3098,  ..., -0.9843, -0.9765, -0.9608],
         ...,
         [-0.7333, -0.6784, -0.6863,  ..., -0.8039, -0.7882, -0.7725],
         [-0.7569, -0.7098, -0.7098,  ..., -0.8039, -0.7569, -0.7412],
         [-0.7647, -0.7804, -0.7333,  ..., -0.7333, -0.8118, -0.7647]],

        [[-0.6314, -0.5608, -0.4745,  ..., -0.9686, -0.9608, -0.9373],
         [-0.7176, -0.6078, -0.4039,  ..., -0.9765, -0.9686, -0.9451],
         [-0.7882, -0.6706, -0.4039,  ..., -0.9843, -0.9765, -0.9608],
         ...,
         [-0.8510, -0.7961, -0.8196,  ..., -0.8196, -0.8039, -0.7961],
         [-0.8902, -0.8353, -0.8196,  ..., -0.8353, -0.8039, -0.7804],
         [-0.8353, -0.8510, -0.8118,  ..., -0.7569, -0.8431, -0.7882]],

        [[-0.6627, -0.6000, -0.5373,  ..., -0.9686, -0.9608, -0.9608],
         [-0.7490, -0.6471, -0.4510,  ..., -